In [ ]:


NDBC_Realtime_Folder = 'https://www.ndbc.noaa.gov/data/realtime2/'
NDBC_Stations_Folder = 'https://www.ndbc.noaa.gov/data/stations/'

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from io import StringIO

station_list = 'https://www.ndbc.noaa.gov/data/stations/station_table.txt'
response = requests.get(station_list)
soup = BeautifulSoup(response.text, 'html.parser')
soup


In [ ]:
# Convert the soup text into a DataFrame
data = StringIO(soup.text)
columns = ['STATION_ID', 'OWNER', 'TTYPE', 'HULL', 'NAME', 'PAYLOAD', 'LOCATION', 'TIMEZONE', 'FORECAST', 'NOTE']
df_stations = pd.read_csv(data, sep='|', names=columns, index_col=False, skiprows=2)
df_stations

In [ ]:
stations_xml = 'https://www.ndbc.noaa.gov/activestations.xml'
response_xml = requests.get(stations_xml)
soup_xml = BeautifulSoup(response_xml.content, 'xml')

soup_xml

stations = []
for station in soup_xml.find_all('station'):
    stations.append(station.attrs)

df_stations = pd.DataFrame(stations)
df_stations

In [ ]:
NDBC_RT_Folder = 'https://www.ndbc.noaa.gov/data/realtime2/'
response_rt = requests.get(NDBC_RT_Folder)
soup_rt = BeautifulSoup(response_rt.text, 'html.parser')

files = [a['href'] for a in soup_rt.find_all('a', href=True)]
files
files = files[5:]
download_links = [NDBC_RT_Folder + file for file in files]
download_links
dataframes = []
for link in download_links:
    print(link)
    st_name = (link.split('/')[-1])
    try:
        response_file = requests.get(link)
        data = StringIO(response_file.text)
        df = pd.read_csv(data, delim_whitespace=True)
        df['STATION_ID'] = st_name.split('.')[0]
        df.to_csv (st_name, index=False)
    except Exception as e:
        print(f"Failed to process {link}: {e}")

dataframes